In [1]:


%matplotlib inline

##THIS IS FOR ACCESSING THE CONCORDE TSP SOLVER
from subprocess import Popen

import os
class tsp:
    def __init__(self, points=None):
        self.points = points if points else []
    def solve(self, filename='_tmp_'):
        with open('%s.dat' % filename, 'w') as fp:
            fp.write('TYPE : TSP\nDIMENSION : %d\n' % len(self.points))
            fp.write('EDGE_WEIGHT_TYPE : GEO\nNODE_COORD_SECTION\n')
            for i, (x, y) in enumerate(self.points):
                fp.write('%d %g %g\n' % (i, x, y))
            fp.write('EOF\n')
        prc = Popen('./concorde   -m %s.dat' % filename, shell=True)
        prc.wait()
        with open('%s.sol' % filename) as fp:
            ll = (''.join(fp)).split()
        self.result = [int(s) for s in ll[1:]]
        for s in ('%s.dat', '%s.sol'): os.remove(s % filename)

            
            

In [2]:
##THIS I FOR ACCESSING THE CONCORDE TSP SOLVER BZCKUP,SWITCHING TO GEO
#from subprocess import Popen
#import os
#class tsp:
#    def __init__(self, points=None):
#        self.points = points if points else []
#    def solve(self, filename='_tmp_'):
#        with open('%s.dat' % filename, 'w') as fp:
#            fp.write('TYPE : TSP\nDIMENSION : %d\n' % len(self.points))
#            fp.write('EDGE_WEIGHT_TYPE : EUC_2D\nNODE_COORD_SECTION\n')
#            for i, (x, y) in enumerate(self.points):
#                fp.write('%d %g %g\n' % (i, x, y))
#            fp.write('EOF\n')
#        prc = Popen('./concorde -m  %s.dat' % filename, shell=True)
#        prc.wait()
#        with open('%s.sol' % filename) as fp:
#            ll = (''.join(fp)).split()
#        self.result = [int(s) for s in ll[1:]]
#        for s in ('%s.dat', '%s.sol'): os.remove(s % filename)

In [3]:
import geopy
import pickle
import json
import pymongo
from geopy.distance import vincenty
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from slugify import slugify
import math
from numbers import Number

# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["bandsintown"]

# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}

import numpy as np
from matplotlib import pyplot as plt


In [ ]:
#INITIALIZE

final_embeddings=[]
totalDists=0
totalOptDists=0
totalOptDistsMin=0
#for artist in db.minedArtists.find({ 'name' : 'David Guetta' }) :
#for artist in db.minedArtists.find({ 'name' : 'Vokab Kompany' }) :
#for artist in db.minedArtists.find({ 'name' : 'Ural Kosaken Chor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Leo Kottke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Björk' }) :
#for artist in db.minedArtists.find({ 'name' : 'Deerhoof' }) :
#for artist in db.minedArtists.find({ 'name' : 'Legends' }) :
#for artist in db.minedArtists.find({ 'name' : 'Laidback Luke' }) :
#for artist in db.minedArtists.find({ 'name' : 'Andy Moor' }) :
#for artist in db.minedArtists.find({ 'name' : 'Mia' }) :
#for artist in db.minedArtists.find({ 'name' : 'Dead Kennedys' }) :
#for artist in db.minedArtists.find({ 'name' : 'House of Pain' }) :
for artist in db.minedArtists.find(no_cursor_timeout=True) :
#LOOP THROUGH ALL ARTISTS IN BASE AND CALCULATE OPTIMIZED PARAMETER
 if os.path.exists("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        print "SKIPPED",artist["name"]
        #print slugify(artist["name"])
        with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          temp= pickle.load( handle)
        #print temp
        tmppp =0
        
        totalArtistTourDist=0
        totalArtistOptTourDist=0
        totalArtistTourDist=temp["totalArtistTourDist"]
        totalArtistOptTourDist=temp["totalArtistOptTourDist"]
        
        totalDists+=totalArtistTourDist
        totalOptDists+=totalArtistOptTourDist
        if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
        else:
            tmppp=float(totalArtistOptTourDist)
        totalOptDistsMin+=tmppp
        #print totalArtistTourDist," / ",totalArtistOptTourDist
        #print totalDists," / ",totalOptDists," / ",totalOptDistsMin," / ",tmppp
        
        #pass
 else:
    

    results={}
    tourindice=0
    results["name"]=artist["name"]
    #print artist["name"]
    totalArtistTourDist=0
    totalArtistOptTourDist=0
    for tour in artist["tours"]:
        #print tour["gigs"][0]["datetime"].year
        #print tour["gigs"][-1]["datetime"].year
        #print int(math.ceil(len(tour["gigs"])/2))
        tourCoords= []
        tourdist=0
        tourdistOpt=0
        tourdistOpt2=0
        tourdistOpt3=0
        tourLength=0
        tourYear=0
        tourYearStart=0
        tourYearEnd=0
        tourYearMid=0
        t=0
        #print len(tour["tourCoords"])
        if len(tour["tourCoords"])>2:
            tourLength=int(math.ceil(len(tour["gigs"])))
            tourYearStart=tour["gigs"][0]["datetime"].year
            tourYearEnd=tour["gigs"][-1]["datetime"].year
            if tourYearStart == tourYearEnd :
                tourYear= tourYearStart
            else:
                tourYearMid= tourYearEnd=tour["gigs"][int(math.ceil(len(tour["gigs"])/2))]["datetime"].year
                if tourYearStart == tourYearMid :
                    tourYear= tourYearStart
                else:
                    tourYear= tourYearEnd
            
            tourindice+=1
            tourCoords=[tuple(tourCoord) for tourCoord in tour["tourCoords"]]
            #print tourCoords
            ###RERUN OPT,PICKING BETTER SOLUTION
            t=tsp(tourCoords)
            t.solve()
            #print "here"

            #print t.result
            #print t.points
            #print "long",len (t.result)
            i = 0
            while i < len (t.result)-1:
                res=t.result[i]
                res2=t.result[i+1]
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdistOpt= tourdistOpt + vincenty(t.points[res],t.points[res2]).kilometers
                
                
                i+=1
            
            j = 0
            while j < len (tourCoords)-1:
                
                #print vincenty(t.points[res],t.points[res2]).kilometers
                tourdist= tourdist + vincenty(tourCoords[j],tourCoords[j+1]).kilometers
                j+=1
            
            #print "DEFAULT TOUR DIST",tourdist
            #print "TOURDISTOPTIMIZED",tourdistOpt
            totalArtistTourDist+=tourdist
            totalArtistOptTourDist+=tourdistOpt
            results[tourindice]={'tourdist':tourdist,'tourdistOpt':tourdistOpt,'tourLength' : tourLength , 'tourYear': tourYear}
    #print results
    tourInd=0
    resultsYears={}
    for tourInd in results:
        #print tourInd
        if tourInd != 'name':
          """"reduce dataset to its relevant years !!!! 
        1990 / Nb de groupes comptabilisés cette année / 1
1991 / Nb de groupes comptabilisés cette année / 1
1992 / Nb de groupes comptabilisés cette année / 3
1993 / Nb de groupes comptabilisés cette année / 1
1994 / Nb de groupes comptabilisés cette année / 2
1995 / Nb de groupes comptabilisés cette année / 2
1996 / Nb de groupes comptabilisés cette année / 2
1997 / Nb de groupes comptabilisés cette année / 8
1998 / Nb de groupes comptabilisés cette année / 2
1999 / Nb de groupes comptabilisés cette année / 15
2000 / Nb de groupes comptabilisés cette année / 4
2001 / Nb de groupes comptabilisés cette année / 1
2002 / Nb de groupes comptabilisés cette année / 3
2003 / Nb de groupes comptabilisés cette année / 7
2004 / Nb de groupes comptabilisés cette année / 10
2005 / Nb de groupes comptabilisés cette année / 17
2006 / Nb de groupes comptabilisés cette année / 22
2007 / Nb de groupes comptabilisés cette année / 2719
2008 / Nb de groupes comptabilisés cette année / 4101
2009 / Nb de groupes comptabilisés cette année / 5447
2010 / Nb de groupes comptabilisés cette année / 7083
2011 / Nb de groupes comptabilisés cette année / 6965
2012 / Nb de groupes comptabilisés cette année / 11048
2013 / Nb de groupes comptabilisés cette année / 13451
2014 / Nb de groupes comptabilisés cette année / 16331
2015 / Nb de groupes comptabilisés cette année / 20790
2016 / Nb de groupes comptabilisés cette année / 25322
2017 / Nb de groupes comptabilisés cette année / 25302
2018 / Nb de groupes comptabilisés cette année / 475
2019 / Nb de groupes comptabilisés cette année / 1)
          if results[tourInd]['tourYear'] >= 2007 and results[tourInd]['tourYear'] <= 2017:""""
            #print results[tourInd]['tourYear']
            if results[tourInd]['tourYear'] not in resultsYears:
              
                resultsYears[results[tourInd]['tourYear']]={'toursCount':0,'toursDist':0,'toursDistOpt':0,'toursLength':0}
            
            resultsYears[results[tourInd]['tourYear']]['toursCount']+=1
            resultsYears[results[tourInd]['tourYear']]['toursDist']=resultsYears[results[tourInd]['tourYear']]['toursDist']+results[tourInd]['tourdist']
            resultsYears[results[tourInd]['tourYear']]['toursDistOpt']=resultsYears[results[tourInd]['tourYear']]['toursDistOpt']+results[tourInd]['tourdistOpt']
            resultsYears[results[tourInd]['tourYear']]['toursLength']=resultsYears[results[tourInd]['tourYear']]['toursLength']+results[tourInd]['tourLength']
            
    #print resultsYears
    statNbYear=len(resultsYears)
    statMeanYearDist=0
    statMeanDistOpt=0
    statMeanYeartoursLength=0
    statMeanYeartoursCount=0
    #Calculating means
    for Year in resultsYears:
        statMeanYearDist+=resultsYears[Year]['toursDist']
        statMeanDistOpt+=resultsYears[Year]['toursDistOpt']
        statMeanYeartoursLength+=resultsYears[Year]['toursLength']
        statMeanYeartoursCount+=resultsYears[Year]['toursCount']
    #print statMeanYearDist
    #print statNbYear
    if statMeanYearDist > 0:
        statMeanYearDist=statMeanYearDist/statNbYear
        statMeanDistOpt=statMeanDistOpt/statNbYear
        statMeanYeartoursLength=statMeanYeartoursLength/statNbYear
        statMeanYeartoursCount=statMeanYeartoursCount/statNbYear
        resultsYears['statMeanYearDist']=statMeanYearDist
        resultsYears['statMeanYearDistOpt']=statMeanDistOpt
        resultsYears['statMeanYeartoursLength']=statMeanYeartoursLength
        resultsYears['statMeanYeartoursCount']=statMeanYeartoursCount
        resultsYears['statNbYear']=statNbYear
        
        for Year in resultsYears:
            if isinstance(Year, Number):
                #print Year
                resultsYears[Year]['stattoursDist']=float(resultsYears[Year]['toursDist']-resultsYears['statMeanYearDist'])/resultsYears['statMeanYearDist']
                resultsYears[Year]['stattoursDistOpt']=float(resultsYears[Year]['toursDistOpt']-resultsYears['statMeanYearDistOpt'])/resultsYears['statMeanYearDistOpt']
                resultsYears[Year]['stattoursLength']=float(resultsYears[Year]['toursLength']-resultsYears['statMeanYeartoursLength'])/resultsYears['statMeanYeartoursLength']
                resultsYears[Year]['stattoursCount']=float(resultsYears[Year]['toursCount']-resultsYears['statMeanYeartoursCount'])/resultsYears['statMeanYeartoursCount']
    
        
        
    #print "TOTAL DEFAULT ARTIST TOURS DISTS",totalArtistTourDist
    #print "TOTAL OPTIMIZED ARTIST TOURS DISTS",totalArtistOptTourDist 
    if totalArtistTourDist > 0 :
        OptArtistPercent=0
        OptArtistPercent=100 *(totalArtistTourDist-totalArtistOptTourDist) / totalArtistTourDist
        results["OptArtistPercent"]=OptArtistPercent
        #print "OPTIMIZED BY ", OptArtistPercent," %"
    #print"_______________________________"
    totalDists+=totalArtistTourDist
    totalOptDists+=totalArtistOptTourDist
    tmppp=0
    if float(totalArtistTourDist) <= float(totalArtistOptTourDist)      :  
            tmppp=float(totalArtistTourDist)
    else:
            tmppp=float(totalArtistOptTourDist)
    totalOptDistsMin+=tmppp
    
    results["resultsYears"]=resultsYears
    results["totalArtistTourDist"]=totalArtistTourDist
    results["totalArtistOptTourDist"]=totalArtistOptTourDist
    
    with open("./tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    #print results['resultsYears']
    #print totalDists," / ",totalArtistTourDist," / ",totalArtistOptTourDist," / ",totalOptDistsMin," / ",tmppp
print "***************************"
print "TOTAL ARTISTS TOURS DISTS",totalDists
print "TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS",totalOptDists 
print "TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS",totalOptDistsMin 
print "AUTO-OPTIMIZED BY ", 100 *(totalDists-totalOptDists) / totalDists," %"
print "MANUALLY OPTIMIZED BY ", 100 *(totalDists-totalOptDistsMin) / totalDists," %"
print "***************************"
with open("./RESULTSGLOBAL.obj", 'wb') as handle:
                            pickle.dump({"totalDists":totalDists,"totalOptDists":totalOptDists,"totalOptDistsMin":totalOptDistsMin,"autoOpt":(totalDists-totalOptDists) / totalDists,"manOpt":(totalDists-totalOptDistsMin) / totalDists}, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#CALCULATE GLOBAL  RAW YEAR STATS
import os

#totalDistsPERYEAR={}
#totalOptDistsPERYEAR={}
#totalOptDistsMinPERYEAR={}
resultsGlobalYears={}
resultsGlobalYearsAddedResultsCount={}
for filename in os.listdir('./tour_miner'):
    with open("./tour_miner/"+filename, 'rb') as handle:
#with open("./tour_miner/young-mc693097c4-17ec-4bb8-9b7c-74a5a2a85723"+".obj", 'rb') as handle:    
        temp= pickle.load( handle)
        resultsYears=temp["resultsYears"]
        #print "------------"
        #print resultsYears  
        #print "------------"
        for Year in resultsYears:
            #print Year
            #print resultsYears[Year]
            
            #print "****"
         if isinstance(Year, Number):
            if Year not in resultsGlobalYears:
                resultsGlobalYears[Year]={"stattoursGlobalLength": 0,"toursGlobalLength": 0,"toursGlobalDistOpt": 0,"stattoursGlobalDist": 0,"toursGlobalDist": 0,"stattoursGlobalDistOpt": 0,"toursGlobalCount": 0,"stattoursGlobalCount": 0,"stattoursGlobalLengthMEAN": 0,"toursGlobalLengthMEAN": 0,"toursGlobalDistOptMEAN": 0,"stattoursGlobalDistMEAN": 0,"toursGlobalDistMEAN": 0,"stattoursGlobalDistOptMEAN": 0,"toursGlobalCountMEAN": 0,"stattoursGlobalCountMEAN": 0,"stattoursGlobalLengthMEAN": 0,"toursGlobalLengthMEAN": 0,"toursGlobalDistOptMEAN": 0,"stattoursGlobalDistMEAN": 0,"toursGlobalDistMEAN": 0,"stattoursGlobalDistOptMEAN": 0,"toursGlobalCountMEAN": 0,"stattoursGlobalCountMEAN": 0}
                resultsGlobalYearsAddedResultsCount[Year]=0
            
                #print "statLE",resultsYears[Year]["stattoursLength"]
            try:
                    resultsGlobalYearsAddedResultsCount[Year]+=1
                    resultsGlobalYears[Year]["stattoursGlobalLength"]=float(resultsYears[Year]["stattoursLength"])+float(resultsGlobalYears[Year]["stattoursGlobalLength"])
                    resultsGlobalYears[Year]["toursGlobalLength"]=float(resultsYears[Year]["toursLength"])+float(resultsGlobalYears[Year]["toursGlobalLength"])
                    resultsGlobalYears[Year]["toursGlobalDistOpt"]=float(resultsYears[Year]["toursDistOpt"])+float(resultsGlobalYears[Year]["toursGlobalDistOpt"])
                    resultsGlobalYears[Year]["stattoursGlobalDist"]=float(resultsYears[Year]["stattoursDist"])+float(resultsGlobalYears[Year]["stattoursGlobalDist"])
                    resultsGlobalYears[Year]["toursGlobalDist"]=float(resultsYears[Year]["toursDist"])+float(resultsGlobalYears[Year]["toursGlobalDist"])
                    resultsGlobalYears[Year]["stattoursGlobalDistOpt"]=float(resultsYears[Year]["stattoursDistOpt"])+float(resultsGlobalYears[Year]["stattoursGlobalDistOpt"])
                    resultsGlobalYears[Year]["toursGlobalCount"]=float(resultsYears[Year]["toursCount"])+float(resultsGlobalYears[Year]["toursGlobalCount"])
                    resultsGlobalYears[Year]["stattoursGlobalCount"]=float(resultsYears[Year]["stattoursCount"])+float(resultsGlobalYears[Year]["stattoursGlobalCount"])
            except KeyError :
                    print Year
                    print resultsYears[Year]
                    print KeyError
                    #NOT ENOUGH DATAS FOR 2018
                    pass
        
        

            
        
    
#WE REALISE THAT TO GET ACCURATE STATS, WE SHOULD FILTER BY YEARS WITH ENOUGH DATAS, AND BANDS WITH ENOUGH SHOWS
    

In [111]:
#print resultsGlobalYears
listP=["","MOY","MAX","MIN","VAR","ECTP"]
for YearG in resultsGlobalYears:
 if YearG >=2007 and YearG <= 2017:
    #CALCULATE MEAN VALUES
    resultsGlobalYears[YearG]["stattoursGlobalLengthMEAN"]=float(resultsGlobalYears[YearG]["stattoursGlobalLength"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["toursGlobalLengthMEAN"]=float(resultsGlobalYears[YearG]["toursGlobalLength"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["toursGlobalDistOptMEAN"]=float(resultsGlobalYears[YearG]["toursGlobalDistOpt"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["stattoursGlobalDistMEAN"]=float(resultsGlobalYears[YearG]["stattoursGlobalDist"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["toursGlobalDistMEAN"]=float(resultsGlobalYears[YearG]["toursGlobalDist"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["stattoursGlobalDistOptMEAN"]=float(resultsGlobalYears[YearG]["stattoursGlobalDistOpt"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["toursGlobalCountMEAN"]=float(resultsGlobalYears[YearG]["toursGlobalCount"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    resultsGlobalYears[YearG]["stattoursGlobalCountMEAN"]=float(resultsGlobalYears[YearG]["stattoursGlobalCount"])/float(resultsGlobalYearsAddedResultsCount[YearG])
    
    #CALCULATE TOT VALUES
    resultsGlobalYears[YearG]["stattoursGlobalLength"]=float(resultsGlobalYears[YearG]["stattoursGlobalLength"])
    resultsGlobalYears[YearG]["toursGlobalLength"]=float(resultsGlobalYears[YearG]["toursGlobalLength"])
    resultsGlobalYears[YearG]["toursGlobalDistOpt"]=float(resultsGlobalYears[YearG]["toursGlobalDistOpt"])
    resultsGlobalYears[YearG]["stattoursGlobalDist"]=float(resultsGlobalYears[YearG]["stattoursGlobalDist"])
    resultsGlobalYears[YearG]["toursGlobalDist"]=float(resultsGlobalYears[YearG]["toursGlobalDist"])
    resultsGlobalYears[YearG]["stattoursGlobalDistOpt"]=float(resultsGlobalYears[YearG]["stattoursGlobalDistOpt"])
    resultsGlobalYears[YearG]["toursGlobalCount"]=float(resultsGlobalYears[YearG]["toursGlobalCount"])
    resultsGlobalYears[YearG]["stattoursGlobalCount"]=float(resultsGlobalYears[YearG]["stattoursGlobalCount"])
    
    

    
    
    print "*----------"
    print YearG,"/ Nb de groupes comptabilisés cette année /",resultsGlobalYearsAddedResultsCount[YearG]
    print "-----------"
    
    
    print "=>stat longueur tournées moy",100*resultsGlobalYears[YearG]["stattoursGlobalLengthMEAN"], "%" 
    print "=>longueur tournées ",resultsGlobalYears[YearG]["toursGlobalLength"]," dates" 
    print "=>longueur tournées moy",resultsGlobalYears[YearG]["toursGlobalLengthMEAN"]," dates" 
    print "=>distances tournées ",resultsGlobalYears[YearG]["toursGlobalDist"]," km" 
    print "=>distances tournées moy",resultsGlobalYears[YearG]["toursGlobalDistMEAN"]," km" 
    print "=>distances tournées optimisés  ",resultsGlobalYears[YearG]["toursGlobalDistOpt"]," km" 
    print "=>distances tournées optimisés moy ",resultsGlobalYears[YearG]["toursGlobalDistOptMEAN"]," km" 
    
    print "=>Stat distances tournées moy",100*resultsGlobalYears[YearG]["stattoursGlobalDistMEAN"]," %" 
    
    print "=>Stat distances tournées optimisés moy",100*resultsGlobalYears[YearG]["stattoursGlobalDistOptMEAN"]," %" 
    print "=>nombre de tournées  ",resultsGlobalYears[YearG]["toursGlobalCount"] 
    print "=>nombre de tournées moy ",resultsGlobalYears[YearG]["toursGlobalCountMEAN"] 
    
    print "=>Stat nombre de tournées moy",100*resultsGlobalYears[YearG]["stattoursGlobalCountMEAN"]," %" 
    
    
    
    
    print "----------*"

*----------
2007 / Nb de groupes comptabilisés cette année / 2719
-----------
=>stat longueur tournées moy -18.4777530356 %
=>longueur tournées  60339.0  dates
=>longueur tournées moy 22.1916145642  dates
=>distances tournées  40780697.6766  km
=>distances tournées moy 14998.4176817  km
=>distances tournées optimisés   35285760.184  km
=>distances tournées optimisés moy  12977.4770813  km
=>Stat distances tournées moy -25.978268979  %
=>Stat distances tournées optimisés moy -24.2540334653  %
=>nombre de tournées   6647.0
=>nombre de tournées moy  2.44464876793
=>Stat nombre de tournées moy 3.95020052891  %
----------*
*----------
2008 / Nb de groupes comptabilisés cette année / 4101
-----------
=>stat longueur tournées moy 3.61238475356 %
=>longueur tournées  113378.0  dates
=>longueur tournées moy 27.6464277006  dates
=>distances tournées  87710447.0051  km
=>distances tournées moy 21387.5754706  km
=>distances tournées optimisés   73997128.0255  km
=>distances tournées optimisés moy 

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:


fig = plt.figure()

ax1 = fig.add_subplot(111)

ax1.set_title("Classement groupes par distances parcourues ")    
ax1.set_xlabel('rang')
ax1.set_ylabel('km')
ax1.plot(data['pos'],data['km'],'x' ,color='r', label="data['nom']")
ax1.set_autoscale_on
leg = ax1.legend()

plt.show()


In [55]:
with open("./tour_miner/"+"young-mc693097c4-17ec-4bb8-9b7c-74a5a2a85723"+".obj", 'rb') as handle:
    temp= pickle.load(handle)
    
print temp

{1: {'tourLength': 3, 'tourdist': 4412.095403569076, 'tourYear': 2013, 'tourdistOpt': 4412.095403569076}, 2: {'tourLength': 3, 'tourdist': 3598.3834783813863, 'tourYear': 2016, 'tourdistOpt': 3598.3834783813863}, 3: {'tourLength': 20, 'tourdist': 27289.99921420611, 'tourYear': 2016, 'tourdistOpt': 9304.647910861806}, 4: {'tourLength': 6, 'tourdist': 4892.130973463658, 'tourYear': 2016, 'tourdistOpt': 4892.130973463658}, 'name': u'Young MC', 6: {'tourLength': 8, 'tourdist': 7787.715648749957, 'tourYear': 2017, 'tourdistOpt': 6323.238414375028}, 7: {'tourLength': 31, 'tourdist': 73680.08716318749, 'tourYear': 2017, 'tourdistOpt': 46773.53267687943}, 8: {'tourLength': 4, 'tourdist': 2438.171813830467, 'tourYear': 2017, 'tourdistOpt': 2438.171813830467}, 9: {'tourLength': 10, 'tourdist': 16354.945789979454, 'tourYear': 2017, 'tourdistOpt': 10940.483749558794}, 10: {'tourLength': 8, 'tourdist': 8978.55589883757, 'tourYear': 2017, 'tourdistOpt': 15295.868189735194}, 'totalArtistOptTourDist':

In [6]:
with open("./RESULTSGLOBAL.obj", 'rb') as handle:
          temp= pickle.load( handle)
print temp

{'manOpt': 0.23255376595961377, 'autoOpt': 0.21995298276047787, 'totalOptDistsMin': 1639279100.4292355, 'totalOptDists': 1666194602.2991636, 'totalDists': 2136018170.0272305}


In [ ]:
***************************
TOTAL ARTISTS TOURS DISTS 2135250690.17
TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS 1665625398.71
TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS 1638787824.49
AUTO-OPTIMIZED BY  21.993918261  %
MANUALLY OPTIMIZED BY  23.2507999161  %
***************************

***************************
TOTAL ARTISTS TOURS DISTS 2136018170.03
TOTAL AUTO OPTIMIZED ARTISTS TOURS DISTS 1666994420.69
TOTAL MANUALLY OPTIMIZED ARTISTS TOURS DISTS 1640164490.57
AUTO-OPTIMIZED BY  21.9578539134  %
MANUALLY OPTIMIZED BY  23.2139260994  %
***************************
